# Analyse des performances de l'entreprise et des tendances globales

In [132]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.cluster import KMeans
import plotly.graph_objects as go
import datetime
import matplotlib.pyplot as plt

##### Importation du csv dans un dataframe

In [133]:
df_ = pd.read_csv('./clean_online_retail.csv')
df = df_.copy()

## Analyse produits

Il peut être intéressant de voir quel est le produit qui a le prix le plus élevé

In [161]:
most_expensive = df.loc[df['Price'] == df['Price'].max()]
most_expensive 

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice,cancelled
748142,C556445,M,Manual,-1,2011-06-10 15:31:00,38970.0,15098,United Kingdom,-38970.0,1


On remarque ici que le produit en question a une description particulière, et surtout que la quantité est de -1, il s'agit certainement d'un remboursement qui aurait été fait manuellement ('Manuel' en description)?

Maintenant intéressons-nous aux produits avec le prix le plus bas.

In [162]:
least_expensive = df.loc[df['Price'] == df['Price'].min()]
least_expensive = least_expensive.groupby(['Description', 'Price'], as_index=False)['Quantity'].agg('sum')
least_expensive 

,Description,Price,Quantity
0,Bank Charges,0.001,1
1,PADS TO MATCH ALL CUSHIONS,0.001,17


On peut voir ici qu'il y a 61 produits dont le prix est affiché à 0, voyons plutôt ce qu'on trouve comme produit si on fait abstraction de ceux qui sont gratuits

In [163]:
df_non_zero_price = df.loc[df['Price'] != 0]
least_expensive_not_null = df_non_zero_price.loc[df_non_zero_price['Price'] == df_non_zero_price['Price'].min()]
least_expensive_not_null = least_expensive_not_null.groupby(['Description', 'Price'], as_index=False)['Quantity'].agg('sum')
least_expensive_not_null

,Description,Price,Quantity
0,Bank Charges,0.001,1
1,PADS TO MATCH ALL CUSHIONS,0.001,17


Cherchons maintenant après les 10 produits les plus vendus et les 10 produits sur lesquels il y a le plus de retours.  

In [164]:
df_products = df.groupby(['Description'], as_index=False)['Quantity'].agg('sum')
df_products_asc = df_products.sort_values('Quantity').head(10)
df_products_asc['Quantity'] = df_products_asc['Quantity'] * (-1)
df_products_desc = df_products.sort_values('Quantity', ascending=False).head(10)

In [165]:
fig = go.Figure(data=[
    go.Bar(name='Produits les plus vendus', 
           x=df_products_desc['Description'].astype(str), 
           y=df_products_desc['Quantity'],
           marker_opacity=1,
           marker={'color': df_products_desc['Quantity'],
                   'colorscale': 'Rainbow'})
    ])

fig.update_layout(title='Produits les plus vendus',
                  title_x=0.45,
                  xaxis_title="Produits",
                  yaxis_title="Quantité vendue",
                  plot_bgcolor='white')
fig.show()

In [166]:
fig = go.Figure(data=[
    go.Bar(name='Produits avec le plus de retours', 
           x=df_products_asc['Description'].astype(str), 
           y=df_products_asc['Quantity'],
           marker_opacity=1,
           marker={'color': df_products_desc['Quantity'],
                   'colorscale': 'Rainbow'}
          )
    ])

fig.update_layout(title='Produits avec le plus de retours',
                  title_x=0.45,
                  title_y=0.97,
                  xaxis_title="Produits",
                  yaxis_title="Quantité retournée",
                  plot_bgcolor='white')
fig.show()

On observe ici que le produit le plus vendu est le 'World War 2 Glider', on voit également que parmis les produits les plus vendus il y a plusieurs boîtes à gâteau.  
A l'inverse ce sont les 'white cherry lights' qui ont fait l'objet du plus grand nombre de retours, parmis les produits les plus retournés on peut voir qu'il y a beaucoup de décorations.

## Analyse clients

### Montant des dépenses

Cherchons maintenant après les 10 clients qui ont dépensé le plus au cours de ces deux ans, ainsi que les 10 qui ont dépensé le moins

In [167]:
df_customers = df.groupby(['Customer ID', 'Country'], as_index=False)['TotalPrice'].agg('sum')
df_customers_asc = df_customers.sort_values('TotalPrice').head(10)
df_customers_desc = df_customers.sort_values('TotalPrice', ascending=False).head(10)

In [168]:
fig = go.Figure(data=[
    go.Bar(name='Clients qui ont le plus acheté', 
           x=df_customers_desc['Customer ID'].astype(str), 
           y=df_customers_desc['TotalPrice'],
           marker_opacity=1,
           hovertext=df_customers_desc['Country'],
           marker={'color': df_customers_desc['TotalPrice'],
                   'colorscale': 'Rainbow'})
    ])

fig.update_layout(title='Clients qui ont le plus acheté',
                  title_x=0.45,
                  xaxis_title="Customer ID",
                  yaxis_title="Montant Total",
                  plot_bgcolor='white')
fig.show()

On peut voir que le client avec l'ID 18102 est celui qui a dépensé le plus ces deux dernières années avec près de 600k, on sait également qu'il vient du Royaume Uni.  
Parmis les 10 meilleurs clients, la plupart viennent du Royaume Uni ou d'Irlande

In [169]:
fig = go.Figure(data=[
    go.Bar(name='Clients qui ont le moins acheté', 
           x=df_customers_asc['Customer ID'].astype(str), 
           y=df_customers_asc['TotalPrice'],
           marker_opacity=1,
           hovertext=df_customers_desc['Country'],
           marker={'color': df_customers_desc['TotalPrice'],
                   'colorscale': 'Rainbow'}
          )
    ])

fig.update_layout(title='Clients qui ont le moins acheté',
                  title_x=0.45,
                  xaxis_title="Customer ID",
                  yaxis_title="Montant Total Remboursé",
                  plot_bgcolor='white')
fig.show()

On peut voir ici que le client avec l'ID 17399 est celui qui a reçu le plus gros remboursement, pour près de 25k.  
Tout comme pour les meilleurs clients, les moins bons ont l'air de venir pour la plupart du Royaume-Uni ainsi que de l'Irlande.

### Panier moyen par client

Cela peut être une feature interessante à utiliser dans le clustering (en plus du montant total des dépenses). Car cela permet de différentier un client qui achète peu souvent mais cher d'un client qui fait très régulièrement des petits paniers. D'un autre côté, l'analyse RFM nous donne déjà cette info si on corrèle la frequence d'achat et le montant.

In [170]:
average_basket_price = df.groupby('Customer ID')['Price'].mean()

average_basket_price.sort_values


<bound method Series.sort_values of Customer ID
12346    10.288889
12347     2.498063
12348     0.697826
12349     4.263220
12350     1.581250
           ...    
18283     1.760768
18284     4.003103
18285     8.350000
18286     4.379286
18287     2.236474
Name: Price, Length: 5921, dtype: float64>

### Analyse par pays

Il pourrait être intéressant de voir maintenant quels sont les pays dont les clients ont le plus et le moins acheté.

In [173]:
df_countries_purchase = df.groupby(['Country'], as_index=False)['TotalPrice'].agg('sum')
df_countries_purchase_asc = df_countries_purchase.sort_values('TotalPrice').head(10)
df_countries_purchase_desc = df_countries_purchase.sort_values('TotalPrice', ascending=False).head(10)

In [174]:
fig = go.Figure(data=[
    go.Bar(name="Pays avec le plus d'achats", 
           x=df_countries_purchase_desc['Country'].astype(str), 
           y=df_countries_purchase_desc['TotalPrice'],
           marker_opacity=1,
           marker={'color': df_countries_purchase_desc['TotalPrice'],
                   'colorscale': 'Rainbow'})
    ])

fig.update_layout(title="Pays avec le plus d'achats",
                  title_x=0.45,
                  xaxis_title="Pays",
                  yaxis_title="Montant Total",
                  plot_bgcolor='white')
fig.show()

In [175]:
fig = go.Figure(data=[
    go.Bar(name="Pays avec le moins d'achats", 
           x=df_countries_purchase_asc['Country'].astype(str), 
           y=df_countries_purchase_asc['TotalPrice'],
           marker_opacity=1,
           marker={'color': df_countries_purchase_desc['TotalPrice'],
                   'colorscale': 'Rainbow'}
          )
    ])

fig.update_layout(title="Pays avec le moins d'achats",
                  title_x=0.45,
                  title_y=0.97,
                  xaxis_title="Pays",
                  yaxis_title="Montant Total",
                  plot_bgcolor='white')
fig.show()

On observe ici que le Royaume Uni est loin devant en terme de dépenses, avec près de 13.5M d'achat soit 24x plus que l'Irlande qui arrive en seconde position.  
A l'inverse, l'Arabie Saoudite est le pays où les clients ont le moins dépensé : seulement 131 € sur ces deux dernières années.  
On peut également être étonné du positionnement de la République Tchèque parmis les pays qui ont le moins dépensé alors que la plupart des autres pays d'Europe sont parmis le top 10.

Nous allons maintenant nous intéresser aux pays où le nombre unique de clients est le plus élevé et ceux où il est le moins élevé.

In [176]:
df_countries_customers = df.groupby(['Country'], as_index=False)['Customer ID'].agg({'Customer ID': 'nunique'})
df_countries_customers.rename(columns={'Customer ID': 'Customer Count'}, inplace=True)
df_countries_customers_asc = df_countries_customers.sort_values('Customer Count').head(10)
df_countries_customers_desc = df_countries_customers.sort_values('Customer Count', ascending=False).head(10)

In [177]:
fig = go.Figure(data=[
    go.Bar(name='Pays avec le plus grand nombre de clients uniques', 
           x=df_countries_customers_desc['Country'].astype(str), 
           y=df_countries_customers_desc['Customer Count'],
           marker_opacity=1,
           marker={'color': df_countries_customers_desc['Customer Count'],
                   'colorscale': 'Rainbow'})
    ])

fig.update_layout(title='Pays avec le plus grand nombre de clients uniques',
                  title_x=0.45,
                  xaxis_title="Pays",
                  yaxis_title="Nombre de clients uniques",
                  plot_bgcolor='white')
fig.show()

In [178]:
fig = go.Figure(data=[
    go.Bar(name='Pays avec le plus petit nombre de clients uniques', 
           x=df_countries_customers_asc['Country'].astype(str), 
           y=df_countries_customers_asc['Customer Count'],
           marker_opacity=1,
           marker={'color': df_countries_customers_desc['Customer Count'],
                   'colorscale': 'Rainbow'}
          )
    ])

fig.update_layout(title='Pays avec le plus petit nombre de clients uniques',
                  title_x=0.45,
                  xaxis_title="Pays",
                  yaxis_title="Nombre de clients uniques",
                  plot_bgcolor='white')
fig.show()

On voit ici que dans le Royaume Uni il y a eu 5410 clients uniques au cours de ces 2 dernières années, ce qui est près de 50x plus que l'Allemagne ou la France qui sont à environ 100 clients uniques.  
Pour ce qui est des pays avec le moins de clients uniques, on peut voir que dans ces 10 pays il n'y a eu qu'un seul client.

### Identifier des grossistes ?

Il faudrait regarder par pays la quantité de produits acheté par client

In [179]:
# Groupby par pays, client et facture, et calculer la somme des quantités positives par facture
country_customer_invoice_quantities = df[df['Quantity'] > 0].groupby(['Country', 'Customer ID', 'Invoice'])['Quantity'].sum()

# Filtrer les données pour les clients ayant plus de 10000 produits par facture
filtered_data = country_customer_invoice_quantities[country_customer_invoice_quantities > 10000]

# Réinitialiser l'index pour accéder aux colonnes "Country", "Customer ID" et "Invoice"
filtered_data = filtered_data.reset_index()

# Groupby par client et compter le nombre d'invoices différentes
customer_invoice_counts = filtered_data.groupby('Customer ID')['Invoice'].nunique()

# Groupby par pays et afficher les identifiants des clients ayant plus de 10000 produits par facture
potential_wholesalers = filtered_data.groupby('Country')['Customer ID'].unique()

# Afficher les potentiels grossistes par pays avec le nombre d'invoices différentes par client
for country, customers in potential_wholesalers.items():
    print(f"Pays : {country}")
    print("Identifiants des clients et nombre d'invoices différentes :")
    for customer in customers:
        invoice_count = customer_invoice_counts[customer]
        print(f"Customer ID : {customer}, Nombre d'invoices différentes : {invoice_count}")
    print()


Pays : Australia
Identifiants des clients et nombre d'invoices différentes :
Customer ID : 12415, Nombre d'invoices différentes : 4

Pays : Denmark
Identifiants des clients et nombre d'invoices différentes :
Customer ID : 13902, Nombre d'invoices différentes : 3

Pays : EIRE
Identifiants des clients et nombre d'invoices différentes :
Customer ID : 14156, Nombre d'invoices différentes : 1

Pays : France
Identifiants des clients et nombre d'invoices différentes :
Customer ID : 14277, Nombre d'invoices différentes : 1

Pays : Netherlands
Identifiants des clients et nombre d'invoices différentes :
Customer ID : 14646, Nombre d'invoices différentes : 7

Pays : United Kingdom
Identifiants des clients et nombre d'invoices différentes :
Customer ID : 12346, Nombre d'invoices différentes : 1
Customer ID : 12931, Nombre d'invoices différentes : 1
Customer ID : 13687, Nombre d'invoices différentes : 1
Customer ID : 13694, Nombre d'invoices différentes : 1
Customer ID : 15838, Nombre d'invoices di

In [180]:
# Seuil de quantité
threshold = 5000

# Filtrer les données pour les clients ayant commandé plus de x fois un même produit
filtered_data = df.groupby(['Country', 'Customer ID', 'StockCode'])['Quantity'].sum().reset_index()
filtered_data = filtered_data[filtered_data['Quantity'] > threshold]

# Compter le nombre d'objets différents commandés plus de x fois par client et par pays
product_counts = filtered_data.groupby(['Country', 'Customer ID'])['StockCode'].nunique().reset_index()

# Filtrer les clients ayant au moins un produit différent commandé plus de x fois
potential_wholesalers = product_counts[product_counts['StockCode'] > 1]

# Afficher les potentiels grossistes par pays avec le nombre d'objets différents commandés plus de x fois
for country in potential_wholesalers['Country'].unique():
    print(f"Pays : {country}")
    print(f"Identifiants des clients et nombre d'objets différents commandés plus de {threshold} fois :")
    customers = potential_wholesalers[potential_wholesalers['Country'] == country]['Customer ID']
    for customer in customers:
        product_count = potential_wholesalers[(potential_wholesalers['Country'] == country) & (potential_wholesalers['Customer ID'] == customer)]['StockCode'].values[0]
        print(f"Customer ID : {customer}, Nombre d'objets différents commandés plus de {threshold} fois : {product_count}")
    print()


Pays : Denmark
Identifiants des clients et nombre d'objets différents commandés plus de 5000 fois :
Customer ID : 13902, Nombre d'objets différents commandés plus de 5000 fois : 13

Pays : Netherlands
Identifiants des clients et nombre d'objets différents commandés plus de 5000 fois :
Customer ID : 14646, Nombre d'objets différents commandés plus de 5000 fois : 6

Pays : United Kingdom
Identifiants des clients et nombre d'objets différents commandés plus de 5000 fois :
Customer ID : 12931, Nombre d'objets différents commandés plus de 5000 fois : 2
Customer ID : 13027, Nombre d'objets différents commandés plus de 5000 fois : 6
Customer ID : 13316, Nombre d'objets différents commandés plus de 5000 fois : 2
Customer ID : 13687, Nombre d'objets différents commandés plus de 5000 fois : 3
Customer ID : 15769, Nombre d'objets différents commandés plus de 5000 fois : 3
Customer ID : 15838, Nombre d'objets différents commandés plus de 5000 fois : 2
Customer ID : 16333, Nombre d'objets différent

In [181]:
# pd.set_option('display.max_rows', None)

customer_id = 13027
customer_data = df[df['Customer ID'] == customer_id]
customer_data.head(10)


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice,cancelled
62873,494930,84992,72 SWEETHEART FAIRY CAKE CASES,240,2010-01-20 10:41:00,0.4,13027,United Kingdom,96.0,0
62874,494930,84991,60 TEATIME FAIRY CAKE CASES,240,2010-01-20 10:41:00,0.4,13027,United Kingdom,96.0,0
62875,494930,21977,PACK OF 60 PINK PAISLEY CAKE CASES,240,2010-01-20 10:41:00,0.4,13027,United Kingdom,96.0,0
62876,494930,21975,PACK OF 60 DINOSAUR CAKE CASES,240,2010-01-20 10:41:00,0.4,13027,United Kingdom,96.0,0
62877,494930,21212,PACK OF 72 RETRO SPOT CAKE CASES,240,2010-01-20 10:41:00,0.4,13027,United Kingdom,96.0,0
65447,495233,21213,PACK OF 72 SKULL CAKE CASES,240,2010-01-22 09:20:00,0.4,13027,United Kingdom,96.0,0
88931,497802,21975,PACK OF 60 DINOSAUR CAKE CASES,240,2010-02-12 13:52:00,0.4,13027,United Kingdom,96.0,0
88932,497802,84992,72 SWEETHEART FAIRY CAKE CASES,240,2010-02-12 13:52:00,0.4,13027,United Kingdom,96.0,0
88933,497802,84991,60 TEATIME FAIRY CAKE CASES,240,2010-02-12 13:52:00,0.4,13027,United Kingdom,96.0,0
88934,497802,21977,PACK OF 60 PINK PAISLEY CAKE CASES,240,2010-02-12 13:52:00,0.4,13027,United Kingdom,96.0,0


Parmis les clients qui commandent en grosse quantité, on n'a pas identifé de potentiels grossistes en étudiant leurs commandes (car il y a aussi des produits commandés à l'unité dans leurs commandes)